In [24]:
import pandas as pd

Greedy

In [25]:
with open("controllers/moose_greedy/results/greedy_performance.csv", "r") as f:
    lines = f.read().splitlines()

rows = [line.strip('"').split(',') for line in lines]
columns = rows[0]
data = rows[1:]

import pandas as pd
greedy = pd.DataFrame(data, columns=columns)

# Converta os dados numéricos
for col in ["Distance", "Distance traveled", "Time", "Avg Speed"]:
    greedy[col] = pd.to_numeric(greedy[col], errors='coerce')

print(greedy.head())
print(greedy["Result"].value_counts())


   Distance  Distance traveled    Time  Avg Speed   Result
0     88.08              90.89  120.58       1.14  Success
1     39.36              41.40   56.78       1.36  Success
2    148.35             160.47  153.36       1.06  Success
3    121.34             125.18  130.37       1.41  Success
4    133.20             139.18  156.91       1.06  Success
Result
Success    80
Failure    20
Name: count, dtype: int64


In [26]:
success_rate = (greedy["Result"] == "Success").mean()
print(f"Taxa de sucesso: {success_rate:.2%}")


Taxa de sucesso: 80.00%


In [27]:
avg_time = greedy["Time"].mean()
print(f"Tempo médio: {avg_time:.2f} segundos")

Tempo médio: 117.41 segundos


In [28]:
greedy["Desvio"] = greedy["Distance traveled"] - greedy["Distance"]
print("Desvio médio (Distância percorrida - planejada):")
print(greedy["Desvio"].describe())

Desvio médio (Distância percorrida - planejada):
count    100.00000
mean      -0.96450
std       20.49203
min      -76.45000
25%        0.95500
50%        3.90000
75%        8.75250
max       37.00000
Name: Desvio, dtype: float64


In [29]:
avg_speed = greedy["Avg Speed"].mean()
print(f"Velocidade média: {avg_speed:.2f} m/s")


Velocidade média: 1.01 m/s


In [30]:
grouped = greedy.groupby("Result").agg({
    "Time": ["mean", "std"],
    "Distance traveled": ["mean", "std"],
    "Avg Speed": ["mean", "std"],
    "Desvio": ["mean", "std"]
})
print("Comparação entre Resultados:")
print(grouped)

Comparação entre Resultados:
             Time            Distance traveled            Avg Speed            \
             mean        std              mean        std      mean       std   
Result                                                                          
Failure   78.5380  68.463487          45.27450  39.502238  0.819000  0.149275   
Success  127.1335  59.401562          96.71825  54.999510  1.059125  0.237093   

           Desvio             
             mean        std  
Result                        
Failure -32.80350  25.267935  
Success   6.99525   7.202722  


In [31]:
corr = greedy["Time"].corr(greedy["Distance traveled"])
print(f"Correlação entre tempo e distância percorrida: {corr:.2f}")


Correlação entre tempo e distância percorrida: 0.90


In [32]:
sucessos = greedy[greedy['Result'] == 'Success']
sucessos['Efficiency (%)'] = (sucessos['Distance'] / sucessos['Distance traveled']) * 100
eficiencia_media_sucesso = sucessos['Efficiency (%)'].mean()

print(f"Eficiência média (sucesso): {eficiencia_media_sucesso:.2f}%")


Eficiência média (sucesso): 96.55%


C:\Users\isabe\AppData\Local\Temp\ipykernel_4736\179856094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucessos['Efficiency (%)'] = (sucessos['Distance'] / sucessos['Distance traveled']) * 100


A*

In [33]:
a_star = pd.read_csv("controllers/moose_astar_supervisor/logs/performance.csv")
a_star.head()

,Distance,Distance traveled,Time,Avg Speed,Battery Used,Height Difference,Completion
0,134.063418,138.036580,371.217591,0.684124,0.834939,5.491311,True
1,64.381674,65.225397,158.566742,0.765357,0.076002,3.887387,True
2,98.954535,105.941125,365.362667,0.815191,0.786068,5.505601,True
3,129.402473,130.279221,244.967768,1.109709,0.871576,6.375358,True
4,82.292162,88.911688,244.983774,0.728626,0.000000,6.470730,True


In [34]:
a_star["Completion"].value_counts()

Completion
False    54
True     46
Name: count, dtype: int64

In [35]:
success_rate = a_star["Completion"].mean()
print(f"Taxa de sucesso: {success_rate:.2%}")

Taxa de sucesso: 46.00%


In [36]:
avg_time = a_star["Time"].mean()
print(f"Tempo médio: {avg_time:.2f} segundos")

Tempo médio: 226.01 segundos


In [37]:
a_star["Desvio"] = a_star["Distance traveled"] - a_star["Distance"]
print("Desvio médio (Distância percorrida - planeada):")
print(a_star["Desvio"].describe())

Desvio médio (Distância percorrida - planeada):
count    100.000000
mean     -40.084466
std       54.724086
min     -183.114938
25%      -85.057155
50%      -13.513878
75%        3.583287
max       16.679392
Name: Desvio, dtype: float64


In [38]:
avg_speed = a_star["Avg Speed"].mean()
print(f"Velocidade média: {avg_speed:.2f} m/s")

Velocidade média: 0.63 m/s


In [39]:
grouped = a_star.groupby("Completion").agg({
    "Time": ["mean", "std"],
    "Distance traveled": ["mean", "std"],
    "Avg Speed": ["mean", "std"],
    "Height Difference": ["mean", "std"]
})

print("Comparação entre Resultados:")
print(grouped)

Comparação entre Resultados:
                  Time             Distance traveled            Avg Speed  \
                  mean         std              mean        std      mean   
Completion                                                                  
False       150.120974  183.903201         24.704365  39.372379  0.625599   
True        315.102704  240.200613         95.138724  47.874324  0.640872   

                     Height Difference            
                 std              mean       std  
Completion                                        
False       0.236304          1.402802  1.998861  
True        0.213160          4.660505  2.663829  


In [40]:
# Filtra só as linhas de sucesso
sucessos = a_star[a_star['Completion'] == True]

# Calcula a eficiência média
sucessos['Efficiency (%)'] = (sucessos['Distance'] / sucessos['Distance traveled']) * 100
eficiencia_media_sucesso = sucessos['Efficiency (%)'].mean()

print(f"Eficiência média (sucesso): {eficiencia_media_sucesso:.2f}%")


Eficiência média (sucesso): 94.66%


C:\Users\isabe\AppData\Local\Temp\ipykernel_4736\1887011073.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucessos['Efficiency (%)'] = (sucessos['Distance'] / sucessos['Distance traveled']) * 100


A2C

In [41]:
a2c = pd.read_csv("controllers/moose_gym/logs/performance.csv")
a2c.head()

,Distance,Distance traveled,Time,Avg Speed,Height Difference,Completion
0,114.352568,187.902079,122.668,1.531794,5.886933,False
1,89.451456,0.039379,112.936,0.000349,0.094197,False
2,76.216701,0.043157,113.148,0.000381,0.207645,False
3,71.014797,0.043159,113.156,0.000381,0.001728,False
4,52.752696,0.032604,40.008,0.000815,0.104459,False


In [42]:
a2c["Completion"].value_counts()

Completion
False    100
Name: count, dtype: int64

In [43]:
avg_time = a2c["Time"].mean()
print(f"Tempo médio: {avg_time:.2f} segundos")

Tempo médio: 63.51 segundos


In [44]:
success_rate = a2c["Completion"].mean()
print(f"Taxa de sucesso: {success_rate:.2%}")

Taxa de sucesso: 0.00%


In [45]:
avg_speed = a2c["Avg Speed"].mean()
print(f"Velocidade média: {avg_speed:.2f} m/s")

Velocidade média: 1.89 m/s


In [46]:
a2c["Desvio"] = a2c["Distance traveled"] - a2c["Distance"]
print("Desvio médio (Distância percorrida - planeada):")
print(a2c["Desvio"].describe())

Desvio médio (Distância percorrida - planeada):
count    100.000000
mean     -22.609669
std       71.580584
min     -140.170285
25%      -81.888407
50%      -31.119599
75%       22.570813
max      173.004203
Name: Desvio, dtype: float64


In [47]:
group = a2c.groupby("Completion").agg({
    "Time": ["mean", "std"],
    "Distance traveled": ["mean", "std"],
    "Avg Speed": ["mean", "std"],
    "Height Difference": ["mean", "std"]})

print("Comparação entre resultados:")
print(group)

Comparação entre resultados:
                Time            Distance traveled            Avg Speed  \
                mean        std              mean        std      mean   
Completion                                                               
False       63.51412  69.548745         79.957577  70.914845  1.894683   

                    Height Difference            
                std              mean       std  
Completion                                       
False       2.35217          3.343846  3.146311  


In [48]:
corr = a2c["Time"].corr(a2c["Distance traveled"])
print(f"Correlação entre tempo e distância percorrida: {corr:.2f}")

Correlação entre tempo e distância percorrida: 0.27


In [49]:
# Select the columns for which to calculate statistics
metrics = ['Distance', 'Distance traveled', 'Time', 'Avg Speed']

# Calculate descriptive statistics for the selected columns
summary_table = a2c[metrics].agg(['mean', 'std', 'min', 'median', 'max'])

# Transpose the table for the desired output format
summary_table = summary_table.T

# Rename the columns for clarity in the report
summary_table = summary_table.rename(columns={'mean': 'Mean', 'std': 'Std Dev', 'min': 'Min', 'median': 'Median', 'max': 'Max'})

# Display the summary table
summary_table

,Mean,Std Dev,Min,Median,Max
Distance,102.567246,41.266988,15.223013,98.107374,203.681626
Distance traveled,79.957577,70.914845,0.032604,82.627596,289.810449
Time,63.514120,69.548745,13.160000,48.156000,678.760000
Avg Speed,1.894683,2.352170,0.000349,1.467422,8.527367


In [50]:
metrics = ['Distance', 'Distance traveled', 'Time', 'Avg Speed', 'Completion']
summary_table = a2c[metrics].agg(['mean', 'std', 'min', 'median', 'max'])
summary_table = summary_table.T
summary_table = summary_table.rename(columns={'mean': 'Mean', 'std': 'Std Dev', 'min': 'Min', 'median': 'Median', 'max': 'Max'})
summary_table

,Mean,Std Dev,Min,Median,Max
Distance,102.567246,41.266988,15.223013,98.107374,203.681626
Distance traveled,79.957577,70.914845,0.032604,82.627596,289.810449
Time,63.51412,69.548745,13.16,48.156,678.76
Avg Speed,1.894683,2.35217,0.000349,1.467422,8.527367
Completion,0.0,0.0,False,0.0,False


Novo AC2

In [51]:
a2c = pd.read_csv("controllers/moose_gym/logs/performance_v5.csv")
a2c.head()

,Distance,Distance traveled,Time,Avg Speed,Height Difference,Height Std,Minimum distance to goal,Completion
0,26.941541,25.319217,14.520,1.743748,3.426072,0.917732,4.999085,True
1,21.009287,18.954831,12.060,1.571711,1.925156,0.444069,4.999577,True
2,23.915260,19.565921,12.548,1.559286,1.654838,0.450806,4.999697,True
3,22.062810,17.567988,11.520,1.524999,1.640841,0.380659,4.988326,True
4,28.609793,31.494882,16.580,1.899571,3.301293,0.774255,4.998841,True


In [52]:
a2c["Completion"].value_counts()

Completion
False    87
True     13
Name: count, dtype: int64

In [53]:
avg_time = a2c["Time"].mean()
print(f"Tempo médio: {avg_time:.2f} segundos")

Tempo médio: 51.85 segundos


In [54]:
success_rate = a2c["Completion"].mean()
print(f"Taxa de sucesso: {success_rate:.2%}")

Taxa de sucesso: 13.00%


In [55]:
avg_speed = a2c["Avg Speed"].mean()
print(f"Velocidade média: {avg_speed:.2f} m/s")

Velocidade média: 1.99 m/s


In [56]:
a2c["Desvio"] = a2c["Distance traveled"] - a2c["Distance"]
print("Desvio médio (Distância percorrida - planeada):")
print(a2c["Desvio"].describe())

Desvio médio (Distância percorrida - planeada):
count    100.000000
mean      60.883028
std      100.940347
min      -33.794218
25%       -9.984259
50%       -0.267503
75%      100.953198
max      295.232922
Name: Desvio, dtype: float64


In [57]:
group = a2c.groupby("Completion").agg({
    "Time": ["mean", "std"],
    "Distance traveled": ["mean", "std"],
    "Avg Speed": ["mean", "std"],
    "Height Difference": ["mean", "std"]})

print("Comparação entre resultados:")
print(group)

Comparação entre resultados:
                 Time            Distance traveled             Avg Speed  \
                 mean        std              mean         std      mean   
Completion                                                                 
False       57.741195  38.406212         97.378496  105.656776  2.040623   
True        12.410462   4.268253         21.356906    8.679138  1.613013   

                     Height Difference            
                 std              mean       std  
Completion                                        
False       2.367026          2.935031  3.467189  
True        0.358466          2.343707  1.300851  


In [58]:
# Select the columns for which to calculate statistics
metrics = ['Distance', 'Distance traveled', 'Time', 'Avg Speed']

# Calculate descriptive statistics for the selected columns
summary_table = a2c[metrics].agg(['mean', 'std', 'min', 'median', 'max'])

# Transpose the table for the desired output format
summary_table = summary_table.T

# Rename the columns for clarity in the report
summary_table = summary_table.rename(columns={'mean': 'Mean', 'std': 'Std Dev', 'min': 'Min', 'median': 'Median', 'max': 'Max'})

# Display the summary table
summary_table

,Mean,Std Dev,Min,Median,Max
Distance,26.612661,9.964789,3.284565,25.891354,47.662261
Distance traveled,87.495689,101.817627,0.002437,23.835766,317.452150
Time,51.848200,38.965466,0.004000,44.120000,159.016000
Avg Speed,1.985034,2.214397,0.200008,1.566946,8.483998


In [59]:
metrics = ['Distance', 'Distance traveled', 'Time', 'Avg Speed', 'Completion']
summary_table = a2c[metrics].agg(['mean', 'std', 'min', 'median', 'max'])
summary_table = summary_table.T
summary_table = summary_table.rename(columns={'mean': 'Mean', 'std': 'Std Dev', 'min': 'Min', 'median': 'Median', 'max': 'Max'})
summary_table

,Mean,Std Dev,Min,Median,Max
Distance,26.612661,9.964789,3.284565,25.891354,47.662261
Distance traveled,87.495689,101.817627,0.002437,23.835766,317.45215
Time,51.8482,38.965466,0.004,44.12,159.016
Avg Speed,1.985034,2.214397,0.200008,1.566946,8.483998
Completion,0.13,0.337998,False,0.0,True


Métricas Globais de Traversabilidade

In [60]:
import pandas as pd

# Suponha que os dados estejam num DataFrame
df = pd.read_csv("controllers/moose_gym/logs/performance_v5.csv")  # ou diretamente do texto acima

# Métricas globais
metrics_global = {
    "Taxa de Sucesso (%)": df["Completion"].mean() * 100,
    "Distância Média Percorrida": df["Distance traveled"].mean(),
    "Desvio Médio da Rota (%)": ((df["Distance traveled"] - df["Distance"]) / df["Distance"]).mean() * 100,
    "Velocidade Média": df["Avg Speed"].mean(),
    "Tempo Médio": df["Time"].mean(),
    "Diferença Média de Altura": df["Height Difference"].mean(),
    "Variabilidade de Altura (std média)": df["Height Std"].mean(),
    "Mínima Distância Média ao Objetivo": df["Minimum distance to goal"].mean(),
}


In [61]:
print("=== MÉTRICAS GLOBAIS DE TRAVERSABILIDADE ===\n")

taxa_sucesso = df["Completion"].mean() * 100
dist_percorrida = df["Distance traveled"].mean()
desvio_rota = ((df["Distance traveled"] - df["Distance"]) / df["Distance"]).mean() * 100
velocidade_media = df["Avg Speed"].mean()
tempo_medio = df["Time"].mean()
altura_media = df["Height Difference"].mean()
altura_std = df["Height Std"].mean()
min_dist_goal = df["Minimum distance to goal"].mean()

print(f"Taxa de Sucesso:         {taxa_sucesso:.2f}%")
print(f"Distância Percorrida:    {dist_percorrida:.2f} m")
print(f"Desvio da Rota:          {desvio_rota:.2f}%")
print(f"Velocidade Média:        {velocidade_media:.2f} m/s")
print(f"Tempo Médio:             {tempo_medio:.2f} s")
print(f"Altura Média:            {altura_media:.2f} m")
print(f"Variabilidade de Altura: {altura_std:.2f} m")
print(f"Distância Mínima Média ao Objetivo: {min_dist_goal:.3f} m")


=== MÉTRICAS GLOBAIS DE TRAVERSABILIDADE ===

Taxa de Sucesso:         13.00%
Distância Percorrida:    87.50 m
Desvio da Rota:          258.27%
Velocidade Média:        1.99 m/s
Tempo Médio:             51.85 s
Altura Média:            2.86 m
Variabilidade de Altura: 0.84 m
Distância Mínima Média ao Objetivo: 22.443 m


Métricas por Mundo

In [62]:
df["Mundo"] = df.index // 10

# Agrupamento por mundo
grouped = df.groupby("Mundo").agg({
    "Completion": lambda x: x.mean() * 100,
    "Distance traveled": "mean",
    "Distance": "mean",
    "Time": "mean",
    "Avg Speed": "mean",
    "Height Difference": "mean",
    "Height Std": "mean",
    "Minimum distance to goal": "mean",
})

# Calcula desvio médio da rota por mundo
grouped["Desvio da Rota (%)"] = ((grouped["Distance traveled"] - grouped["Distance"]) / grouped["Distance"]) * 100

print("\n=== MÉTRICAS POR MUNDO ===\n")
for mundo, row in grouped.iterrows():
    print(f"--- Mundo {mundo} ---")
    print(f"  Taxa de Sucesso:         {row['Completion']:.2f}%")
    print(f"  Distância Percorrida:    {row['Distance traveled']:.2f} m")
    print(f"  Desvio da Rota:          {row['Desvio da Rota (%)']:.2f}%")
    print(f"  Velocidade Média:        {row['Avg Speed']:.2f} m/s")
    print(f"  Tempo Médio:             {row['Time']:.2f} s")
    print(f"  Altura Média:            {row['Height Difference']:.2f} m")
    print(f"  Altura Std:              {row['Height Std']:.2f} m")
    print(f"  Dist. Mín. ao Objetivo:  {row['Minimum distance to goal']:.3f} m\n")



=== MÉTRICAS POR MUNDO ===

--- Mundo 0 ---
  Taxa de Sucesso:         90.00%
  Distância Percorrida:    43.98 m
  Desvio da Rota:          68.87%
  Velocidade Média:        1.72 m/s
  Tempo Médio:             21.40 s
  Altura Média:            2.78 m
  Altura Std:              0.75 m
  Dist. Mín. ao Objetivo:  7.534 m

--- Mundo 1 ---
  Taxa de Sucesso:         20.00%
  Distância Percorrida:    77.43 m
  Desvio da Rota:          162.22%
  Velocidade Média:        2.34 m/s
  Tempo Médio:             35.18 s
  Altura Média:            2.62 m
  Altura Std:              0.78 m
  Dist. Mín. ao Objetivo:  24.541 m

--- Mundo 2 ---
  Taxa de Sucesso:         0.00%
  Distância Percorrida:    9.83 m
  Desvio da Rota:          -53.92%
  Velocidade Média:        0.20 m/s
  Tempo Médio:             49.05 s
  Altura Média:            1.32 m
  Altura Std:              0.20 m
  Dist. Mín. ao Objetivo:  20.359 m

--- Mundo 3 ---
  Taxa de Sucesso:         0.00%
  Distância Percorrida:    67.04 m
  D

In [63]:
# Normalizando Altura Std entre 0 e 100
from sklearn.preprocessing import MinMaxScaler

grouped["Traversability"] = 1 - MinMaxScaler().fit_transform(grouped[["Height Std"]])
grouped["Traversability"] *= 100  # escala percentual

# Exibir em formato de tabela
print("\nSUCCESS RATE FOR ROBOT REACHING GOAL TARGET LOCATIONS\n")
print(f"{'Mundo':<10}{'Traversability':<20}{'Success Rate (%)':<20}")
for idx, row in grouped.iterrows():
    print(f"{idx:<10}{row['Traversability']:<20.1f}{row['Completion']:<20.1f}")



SUCCESS RATE FOR ROBOT REACHING GOAL TARGET LOCATIONS

Mundo     Traversability      Success Rate (%)    
0         68.7                90.0                
1         67.1                20.0                
2         99.5                0.0                 
3         87.7                0.0                 
4         12.1                0.0                 
5         65.1                10.0                
6         84.3                0.0                 
7         100.0               0.0                 
8         0.0                 10.0                
9         53.6                0.0                 


In [64]:
from sklearn.preprocessing import MinMaxScaler

# Normaliza a altura padrão (quanto menor o desnível, maior a traversabilidade)
traversabilidade = 1 - MinMaxScaler().fit_transform(df[["Height Std"]])
df["Traversability"] = traversabilidade * 100  # escala percentual

# Calcula a média global
traversabilidade_global = df["Traversability"].mean()
taxa_sucesso_global = df["Completion"].mean() * 100

# Exibe o resultado
print("\nSUCCESS RATE AND TRAVERSABILITY (GLOBAL)\n")
print(f"{'Global':<10}{'Traversability':<20}{'Success Rate (%)':<20}")
print(f"{'':<10}{traversabilidade_global:<20.1f}{taxa_sucesso_global:<20.1f}")



SUCCESS RATE AND TRAVERSABILITY (GLOBAL)

Global    Traversability      Success Rate (%)    
          81.2                13.0                
